In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [72]:
location = 'binhduong'

file_name = "./dataset/" + location + ".csv";
df = pd.read_csv(file_name)
df.rename(columns={'MO': 'MONTH', 'DY': 'DAY'}, inplace=True)
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
df = df.drop(columns=['YEAR', 'MONTH', 'DAY',]).set_index('DATE')
df.head()

,T2M
DATE,
1981-01-10,27.39
1981-01-11,25.19
1981-01-12,24.27
1981-01-13,24.74
1981-01-14,26.12


In [73]:
import numpy
def create_dataset(dataset, look_back=3):
 dataX, dataY = [], []
 for i in range(len(dataset)-look_back-1):
  a = dataset[i:(i+look_back)]
  dataX.append(a)
  dataY.append(dataset[i + look_back])
 return numpy.array(dataX), numpy.array(dataY)

# Thiết lập seed cho random để có thể tái tạo lại kết quả
numpy.random.seed(365)

# Load dataset
product_name = "T2M"
print("Dataframe: \n", df)
dataset = np.array(df.T2M.values.astype('float32'))# Chuyển đổi dataframe thành numpy array

# # Chuẩn hóa dataset
# scaler = MinMaxScaler(feature_range=(0, 1)) # Tạo MinMaxScaler để chuẩn hóa dữ liệu về phạm vi từ 0 đến 1
# dataset = scaler.fit_transform(dataset) # Chuẩn hóa dataset
# print("Dataset after scale: \n", dataset)

# Chia dataset thành tập train và test
train_size = int(len(dataset) * 0.7) # 90% dữ liệu dùng để huấn luyện
test_size = len(dataset) - train_size # 10% dữ liệu dùng để kiểm thử
train, test = dataset[0:train_size], dataset[train_size:] # Chia dataset

# Reshape dữ liệu thành X=t và Y=t+1 (input và output)
look_back = 365
trainX, trainY = create_dataset(train, look_back) # Tạo dữ liệu huấn luyện
testX, testY = create_dataset(test, look_back) # Tạo dữ liệu kiểm thử
print("TrainX shape: ", trainX.shape)
print("TrainY shape: ", trainY.shape)

Dataframe: 
               T2M
DATE             
1981-01-10  27.39
1981-01-11  25.19
1981-01-12  24.27
1981-01-13  24.74
1981-01-14  26.12
...           ...
2023-02-25  26.62
2023-02-26  26.80
2023-02-27  27.72
2023-02-28  29.12
2023-03-01  28.97

[15391 rows x 1 columns]
TrainX shape:  (10407, 365)
TrainY shape:  (10407,)


In [74]:
from sklearn.linear_model import BayesianRidge

reg = BayesianRidge(tol=1e-6, fit_intercept=False, compute_score=True)
reg.fit(trainX, trainY)

BayesianRidge(compute_score=True, fit_intercept=False, tol=1e-06)

In [75]:
import joblib


joblib.dump(reg, './models/bayesian_ridge_' + location + '_model.pkl')
# ymean, ystd = reg.predict(x_test, return_std=True)

['./models/bayesian_ridge_binhduong_model.pkl']

In [76]:
ytestmean, yteststd = reg.predict(testX, return_std=True)

In [77]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(testY, ytestmean)
mse_bayes = mean_squared_error(list(ytestmean), testY)
rmse_bayes = mean_squared_error(list(ytestmean), testY)**0.5

In [78]:
print("Mean Absolute Error:      ", mae)
print("Mean Squared Error:       ", mse_bayes)
print("Root Mean Squared Error:  ", rmse_bayes)


Mean Absolute Error:       0.58908606
Mean Squared Error:        0.6122717
Root Mean Squared Error:   0.7824779397091264


In [79]:
bayes_model = joblib.load("bayesian_ridge_model.pkl")
# Assuming df.T2M is a list-like object containing the data
last_items = df.T2M[-90:]
# Using the Bayesian model to predict based on the last 30 items

prediction = bayes_model.predict(np.array([last_items]))

print(prediction)

In [80]:
# # Find the last date in the 'Date' column of your DataFrame
# last_date = df.index.max()
# last_date
# # Generate a date range from the day after the last date until today
# date_range = pd.date_range(start=last_date + pd.DateOffset(days=1), end=pd.Timestamp.today(), freq='D')
# date_range

In [81]:
# predicted_values = []

# # Get the last 30 items from the 'T2M' column in your DataFrame
# predicted_values = df['T2M'].tail(90).values.tolist()
# last_items = df.T2M[-90:].values.tolist()
# # List to store the new predictions along with their dates
# new_predict = []

# # Predict for each date and update predicted_values
# for date, value in date_range:
#     prediction = bayes_model.predict([predicted_values])
#     predicted_values.append(prediction[0])
#     new_predict.append((date, round(prediction[0], 0)))  # Storing date along with rounded prediction
#     # Remove the first element to keep the list size at 90
#     predicted_values.pop(0)

# new_predict

In [82]:
# # Generate a date range from the day after the last date until today
# date_range = pd.date_range(start=last_date + pd.DateOffset(days=1), end=pd.Timestamp.today(), freq='D')
# date_range

In [83]:
# print("All Predicted Values:", predicted_values)